# Demo Pipeline

In this workbook we will be demonstrating how to use streamsight to create
a pipeline to evaluate RecSys algorithms. Please refer to [demo.ipynb](demo.ipynb)
if you have not as it outlines how the pipeline works under the hood.

We will use Amazon movie data to show case the pipeline and some of the common
methods that you can call to evaluate your RecSys algorithms.

To start off, we will set the k+100 value to be 10 first. This will mean that
for any of the top K metric or algorithm, we will only consider the top 10
recommendations.

In [1]:
k = 10

## Load

We will load the dataset of choice and instantiate the setting that we want
to evaluate the algorithms on. We will use the sliding window setting for this
demo to show case the results of the evaluation.

Similarly to the demo, we can specify a range of parametres to create different
window sizes.

In [2]:
from streamsight.datasets import AmazonMovieDataset
from streamsight.settings import SlidingWindowSetting

dataset = AmazonMovieDataset(use_default_filters=False)
data = dataset.load()
setting_sliding = SlidingWindowSetting(
    background_t=1530000000,
    window_size=60 * 60 * 24 * 30, # day times N
    n_seq_data=1,
    top_K=k
)
setting_sliding.split(data)


INFO     - streamsight - Logging started
WARNING  - streamsight - Logging warning works
WARNING  - py.warnings - /Users/ngtzekean/Github/Streamsight/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

INFO     - streamsight.datasets.base - AmazonMovieDataset is loading dataset...
INFO     - streamsight.datasets.base - AmazonMovieDataset dataset loaded - Took 12.4s
INFO     - streamsight.settings.base - Splitting data...


4it [00:01,  3.89it/s]                       

INFO     - streamsight.settings.sliding_window_setting - Finished split with window size 2592000 seconds. Number of splits: 4


## Evaluate

The evaluation of the algorithm will be abstracted by the pipeline. To create
the pipeline a builder class is used. This is the recommended way to create
pipelines as it allows for easy modification of the pipeline and easy
reproduction of the pipeline.

Adding of algorithm and metric can be done as shown below. Once the builder is
set up, the pipeline can will be returned by calling the `build` method.

Running the pipeline can be done via the `run` method. To run the pipeline in
step, the `run_step` method can be used. This will run the pipeline step by
step.

In [3]:
from streamsight.evaluators import EvaluatorBuilder

builder = EvaluatorBuilder(ignore_unknown_item=True,
                           ignore_unknown_user=True)
builder.add_setting(setting_sliding)

builder.add_algorithm("ItemKNNStatic", {"K": k})
builder.add_algorithm("ItemKNNRolling", {"K": k})
builder.add_algorithm("ItemKNNIncremental", {"K": k})
builder.add_algorithm("Random", {"K": k})

builder.add_metric("PrecisionK", K=k)
builder.add_metric("RecallK", K=k)
evaluator = builder.build()

evaluator.run()

INFO     - streamsight.evaluators.evaluator_pipeline - Phase 1: Preparing the evaluator...
INFO     - streamsight.algorithms.base - Fitting ItemKNNStatic complete - Took 7.94s
INFO     - streamsight.algorithms.base - Fitting ItemKNNRolling complete - Took 7.14s
INFO     - streamsight.algorithms.itemknn_incremental - Updating historical data for ItemKNNIncremental - Took 0.0343s
INFO     - streamsight.algorithms.base - Fitting ItemKNNIncremental complete - Took 7.95s
INFO     - streamsight.algorithms.base - Fitting Random complete - Took 0.246s
INFO     - streamsight.settings.base - Resetting data generators.
INFO     - streamsight.settings.base - Data generators are reset.


  0%|          | 0/4 [00:00<?, ?it/s]

INFO     - streamsight.evaluators.evaluator_pipeline - Phase 2: Evaluating the algorithms...
INFO     - streamsight.evaluators.evaluator_pipeline - Phase 3: Releasing the data...
INFO     - streamsight.algorithms.base - Fitting ItemKNNStatic complete - Took 0.727s
INFO     - streamsight.algorithms.base - Fitting ItemKNNRolling complete - Took 0.72s
INFO     - streamsight.algorithms.itemknn_incremental - Updating historical data for ItemKNNIncremental - Took 0.0848s
INFO     - streamsight.algorithms.base - Fitting ItemKNNIncremental complete - Took 8.22s
INFO     - streamsight.algorithms.base - Fitting Random complete - Took 0.0239s


 25%|██▌       | 1/4 [00:15<00:47, 15.69s/it]

INFO     - streamsight.evaluators.evaluator_pipeline - Phase 2: Evaluating the algorithms...
INFO     - streamsight.evaluators.evaluator_pipeline - Phase 3: Releasing the data...
INFO     - streamsight.algorithms.base - Fitting ItemKNNStatic complete - Took 0.734s
INFO     - streamsight.algorithms.base - Fitting ItemKNNRolling complete - Took 0.723s
INFO     - streamsight.algorithms.itemknn_incremental - Updating historical data for ItemKNNIncremental - Took 0.0893s
INFO     - streamsight.algorithms.base - Fitting ItemKNNIncremental complete - Took 8.22s
INFO     - streamsight.algorithms.base - Fitting Random complete - Took 0.0234s


 50%|█████     | 2/4 [00:31<00:31, 15.70s/it]

INFO     - streamsight.evaluators.evaluator_pipeline - Phase 2: Evaluating the algorithms...
INFO     - streamsight.evaluators.evaluator_pipeline - Phase 3: Releasing the data...
INFO     - streamsight.algorithms.base - Fitting ItemKNNStatic complete - Took 0.736s
INFO     - streamsight.algorithms.base - Fitting ItemKNNRolling complete - Took 0.752s
INFO     - streamsight.algorithms.itemknn_incremental - Updating historical data for ItemKNNIncremental - Took 0.14s
INFO     - streamsight.algorithms.base - Fitting ItemKNNIncremental complete - Took 8.09s
INFO     - streamsight.algorithms.base - Fitting Random complete - Took 0.0231s


 75%|███████▌  | 3/4 [00:47<00:15, 15.68s/it]

INFO     - streamsight.evaluators.evaluator_pipeline - Phase 2: Evaluating the algorithms...
INFO     - streamsight.evaluators.evaluator_pipeline - Phase 3: Releasing the data...
INFO     - streamsight.algorithms.base - Fitting ItemKNNStatic complete - Took 0.726s
INFO     - streamsight.algorithms.base - Fitting ItemKNNRolling complete - Took 0.776s
INFO     - streamsight.algorithms.itemknn_incremental - Updating historical data for ItemKNNIncremental - Took 0.0956s
INFO     - streamsight.algorithms.base - Fitting ItemKNNIncremental complete - Took 8.2s
INFO     - streamsight.algorithms.base - Fitting Random complete - Took 0.0146s


100%|██████████| 4/4 [01:02<00:00, 15.66s/it]


## Metric Results
The following metrics are calculated for each algorithm in the various splits.

We define the micro metric as the computation of all users and items in that
particular window.

We define the macro metric as the computation of all users and items across all
windows where the confusion matrix is first summed across all windows before
computing the metric.

In [4]:
evaluator.metric_results(level="micro")

score  num_user
Algorithm                    Timestamp    Metric                           
ItemKNNStatic(K=10)          t=1530000000 PrecisionK_10  0.002425      4289
                                          RecallK_10     0.017662      4289
                             t=1532592000 PrecisionK_10  0.000513      2532
                                          RecallK_10     0.003220      2532
                             t=1535184000 PrecisionK_10  0.000905       884
                                          RecallK_10     0.009050       884
                             t=1537776000 PrecisionK_10  0.000000        44
                                          RecallK_10     0.000000        44
ItemKNNRolling(K=10)         t=1530000000 PrecisionK_10  0.002425      4289
                                          RecallK_10     0.017662      4289
                             t=1532592000 PrecisionK_10  0.000513      2532
                                          RecallK_10     0.003220      2532
                             t=1535184000 PrecisionK_10  0.000905       884
                                          RecallK_10     0.009050       884
                             t=1537776000 PrecisionK_10  0.000000        44
                                          RecallK_10     0.000000        44
ItemKNNIncremental(K=10)     t=1530000000 PrecisionK_10  0.002425      4289
                                          RecallK_10     0.017662      4289
                             t=1532592000 PrecisionK_10  0.002093      2532
                                          RecallK_10     0.016733      2532
                             t=1535184000 PrecisionK_10  0.001697       884
                                          RecallK_10     0.014140       884
                             t=1537776000 PrecisionK_10  0.002273        44
                                          RecallK_10     0.011364        44
Random(K=10,seed=2147483648) t=1530000000 PrecisionK_10  0.000000      4289
                                          RecallK_10     0.000000      4289
                             t=1532592000 PrecisionK_10  0.000000      2532
                                          RecallK_10     0.000000      2532
                             t=1535184000 PrecisionK_10  0.000000       884
                                          RecallK_10     0.000000       884
                             t=1537776000 PrecisionK_10  0.000000        44
                                          RecallK_10     0.000000        44

In [5]:
evaluator.metric_results(level="macro")

score
Algorithm                    Metric                 
ItemKNNStatic(K=10)          PrecisionK_10  0.001613
                             RecallK_10     0.011860
ItemKNNRolling(K=10)         PrecisionK_10  0.001613
                             RecallK_10     0.011860
ItemKNNIncremental(K=10)     PrecisionK_10  0.002233
                             RecallK_10     0.016921
Random(K=10,seed=2147483648) PrecisionK_10  0.000000
                             RecallK_10     0.000000

To run the pipeline without ignoring unknown items and users, set ignore_unknown_item
and ignore_unknown_user to False. The default value for these parameters is True.

Below we can see the results of the evaluation for the different algorithms. The evaluation
results are shown for the micro and macro levels. Note that the number of users being
evaluated on would be now more than the case where the unknown items and users are ignored.

Note that there will be a need to load the dataset again as there would be modification
of the data in the setting object when the evaluator is executed.

In [6]:
from streamsight.datasets import AmazonMovieDataset
from streamsight.settings import SlidingWindowSetting

dataset = AmazonMovieDataset(use_default_filters=False)
data = dataset.load()
setting_sliding = SlidingWindowSetting(
    background_t=1530000000,
    window_size=60 * 60 * 24 * 30, # day times N
    n_seq_data=1,
    top_K=k
)
setting_sliding.split(data)


from streamsight.evaluators import EvaluatorBuilder

builder = EvaluatorBuilder(False, False)
builder.add_setting(setting_sliding)

builder.add_algorithm("ItemKNNStatic", {"K": k})
builder.add_algorithm("ItemKNNRolling", {"K": k})
builder.add_algorithm("ItemKNNIncremental", {"K": k})
builder.add_algorithm("Random", {"K": k})

builder.add_metric("PrecisionK", K=k)
builder.add_metric("RecallK", K=k)
evaluator = builder.build()

evaluator.run()

INFO     - streamsight.datasets.base - AmazonMovieDataset is loading dataset...
INFO     - streamsight.datasets.base - AmazonMovieDataset dataset loaded - Took 12.9s
INFO     - streamsight.settings.base - Splitting data...


4it [00:00,  4.13it/s]                       

INFO     - streamsight.settings.sliding_window_setting - Finished split with window size 2592000 seconds. Number of splits: 4


INFO     - streamsight.evaluators.evaluator_pipeline - Phase 1: Preparing the evaluator...
INFO     - streamsight.algorithms.base - Fitting ItemKNNStatic complete - Took 9.22s
INFO     - streamsight.algorithms.base - Fitting ItemKNNRolling complete - Took 9.1s
INFO     - streamsight.algorithms.itemknn_incremental - Updating historical data for ItemKNNIncremental - Took 0.048s
INFO     - streamsight.algorithms.base - Fitting ItemKNNIncremental complete - Took 9.17s
INFO     - streamsight.algorithms.base - Fitting Random complete - Took 0.255s
INFO     - streamsight.settings.base - Resetting data generators.
INFO     - streamsight.settings.base - Data generators are reset.


  0%|          | 0/4 [00:00<?, ?it/s]

...

 50%|█████     | 2/4 [00:32<00:32, 16.41s/it]

INFO     - streamsight.evaluators.evaluator_pipeline - Phase 2: Evaluating the algorithms...
INFO     - streamsight.evaluators.evaluator_pipeline - Phase 3: Releasing the data...
INFO     - streamsight.algorithms.base - Fitting ItemKNNStatic complete - Took 0.74s
INFO     - streamsight.algorithms.base - Fitting ItemKNNRolling complete - Took 0.734s
INFO     - streamsight.algorithms.itemknn_incremental - Updating historical data for ItemKNNIncremental - Took 0.101s
INFO     - streamsight.algorithms.base - Fitting ItemKNNIncremental complete - Took 7.22s
INFO     - streamsight.algorithms.base - Fitting Random complete - Took 0.0234s


 75%|███████▌  | 3/4 [00:47<00:15, 15.52s/it]

INFO     - streamsight.evaluators.evaluator_pipeline - Phase 2: Evaluating the algorithms...
INFO     - streamsight.evaluators.evaluator_pipeline - Phase 3: Releasing the data...
INFO     - streamsight.algorithms.base - Fitting ItemKNNStatic complete - Took 0.712s
INFO     - streamsight.algorithms.base - Fitting ItemKNNRolling complete - Took 0.709s
INFO     - streamsight.algorithms.itemknn_incremental - Updating historical data for ItemKNNIncremental - Took 0.0825s
INFO     - streamsight.algorithms.base - Fitting ItemKNNIncremental complete - Took 7.47s
INFO     - streamsight.algorithms.base - Fitting Random complete - Took 0.0126s


100%|██████████| 4/4 [01:01<00:00, 15.45s/it]


In [7]:
evaluator.metric_results(level="micro")

score  num_user
Algorithm                    Timestamp    Metric                           
ItemKNNStatic(K=10)          t=1530000000 PrecisionK_10  0.001136      9157
                                          RecallK_10     0.008218      9157
                             t=1532592000 PrecisionK_10  0.000229      5667
                                          RecallK_10     0.001439      5667
                             t=1535184000 PrecisionK_10  0.000432      1851
                                          RecallK_10     0.004322      1851
                             t=1537776000 PrecisionK_10  0.000000       119
                                          RecallK_10     0.000000       119
ItemKNNRolling(K=10)         t=1530000000 PrecisionK_10  0.001136      9157
                                          RecallK_10     0.008218      9157
                             t=1532592000 PrecisionK_10  0.000229      5667
                                          RecallK_10     0.001439      5667
                             t=1535184000 PrecisionK_10  0.000432      1851
                                          RecallK_10     0.004322      1851
                             t=1537776000 PrecisionK_10  0.000000       119
                                          RecallK_10     0.000000       119
ItemKNNIncremental(K=10)     t=1530000000 PrecisionK_10  0.001136      9157
                                          RecallK_10     0.008218      9157
                             t=1532592000 PrecisionK_10  0.000935      5667
                                          RecallK_10     0.007476      5667
                             t=1535184000 PrecisionK_10  0.000810      1851
                                          RecallK_10     0.006753      1851
                             t=1537776000 PrecisionK_10  0.000840       119
                                          RecallK_10     0.004202       119
Random(K=10,seed=2064682643) t=1530000000 PrecisionK_10  0.000011      9157
                                          RecallK_10     0.000011      9157
                             t=1532592000 PrecisionK_10  0.000000      5667
                                          RecallK_10     0.000000      5667
                             t=1535184000 PrecisionK_10  0.000000      1851
                                          RecallK_10     0.000000      1851
                             t=1537776000 PrecisionK_10  0.000000       119
                                          RecallK_10     0.000000       119

In [8]:
evaluator.metric_results(level="macro")

score
Algorithm                    Metric                 
ItemKNNStatic(K=10)          PrecisionK_10  0.000744
                             RecallK_10     0.005443
ItemKNNRolling(K=10)         PrecisionK_10  0.000744
                             RecallK_10     0.005443
ItemKNNIncremental(K=10)     PrecisionK_10  0.001030
                             RecallK_10     0.007778
Random(K=10,seed=2064682643) PrecisionK_10  0.000006
                             RecallK_10     0.000006